In [ ]:
import pandas as pd
moviesdf=pd.read_csv('/content/drive/MyDrive/Data/ratings.csv')


In [ ]:
moviesdf.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [ ]:
moviesdf=moviesdf[1:1000].pivot(index='userId',columns='movieId',values='rating')
print(moviesdf.shape)
moviesdf.head()

(15, 775)


movieId,1,5,6,7,11,16,17,25,29,32,...,96821,98809,99114,106489,106920,112552,116797,122904,134853,168252
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,0.5,4.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN
2,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from numpy import NaN
import numpy as np
# movies_already_predictedby1=[i for i in moviesdf.iloc[1] if not np.isnan(i)]
movies_already_predictedby1=[]
cnt=0
for i in moviesdf.columns:
  val=moviesdf.loc[1][i]
  if not np.isnan(val):movies_already_predictedby1.append(i)
  if val>3:cnt+=1
len(movies_already_predictedby1),cnt,movies_already_predictedby1

(26,
 24,
 [147,
  858,
  1221,
  1246,
  1968,
  2762,
  2918,
  2959,
  4226,
  4878,
  5577,
  33794,
  54503,
  58559,
  59315,
  68358,
  69844,
  73017,
  81834,
  91500,
  91542,
  92439,
  96821,
  98809,
  99114,
  112552])

In [ ]:
num_features=10
moviesdf.fillna(0,inplace=True)
moviesdf.head(2)

movieId,1,5,6,7,11,16,17,25,29,32,...,96821,98809,99114,106489,106920,112552,116797,122904,134853,168252
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.5,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
moviematrix=moviesdf.to_numpy()
moviematrix[:2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 3., 0., ..., 0., 0., 0.]])

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
moviessparse=csr_matrix(moviematrix)
u,sigma,vt=svds(moviessparse,k=num_features)


In [ ]:
sigma = np.diag(sigma)
u.shape,vt.shape,sigma.shape

((15, 10), (10, 775), (10, 10))

In [ ]:
print(np.dot(sigma,vt).shape)
newmatrix=np.dot(u,np.dot(sigma,vt))
newmatrix.shape

(10, 775)


(15, 775)

In [ ]:
newmatrixnorm=(newmatrix-newmatrix.min())/(newmatrix.max()-newmatrix.min())

In [ ]:
new_movieratings=pd.DataFrame(newmatrix,columns=moviesdf.columns,index=moviesdf.index)
new_movieratings.head()

movieId,1,5,6,7,11,16,17,25,29,32,...,96821,98809,99114,106489,106920,112552,116797,122904,134853,168252
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000549,-0.001363,0.003562,-0.014932,-0.005804,-0.000494,-0.000494,-0.001363,-0.000618,-0.003370,...,4.975353,0.499868,3.981319,0.002074,0.001814,4.975353,0.002333,0.000518,0.001296,0.023731
2,0.004621,2.992575,0.003606,0.007244,-0.003411,0.004210,0.004210,2.992575,0.005262,2.003038,...,-0.002272,-0.002686,-0.002910,-0.002185,-0.001912,-0.002272,-0.002458,-0.000546,-0.001366,-0.022844
3,0.004532,-0.011290,-0.042919,0.244620,0.004901,0.023927,0.023927,-0.011290,0.029909,0.059893,...,0.340143,-0.040127,0.239163,-0.065904,-0.057666,0.340143,-0.074142,-0.016476,-0.041190,-0.008255
4,0.001062,-0.001321,0.001920,-0.002134,-0.003840,0.000581,0.000581,-0.001321,0.000726,-0.000338,...,-0.005223,0.002023,-0.003047,0.002263,0.001980,-0.005223,0.002546,0.000566,0.001414,-0.030744
5,-0.003482,0.007495,0.000583,2.973953,0.069382,-0.005820,-0.005820,0.007495,-0.007275,-0.007029,...,-0.022742,0.009853,-0.012804,0.010780,0.009433,-0.022742,0.012128,0.002695,0.006738,-0.041323


In [ ]:
new_movieratings=new_movieratings.T
new_movieratings.head(3)

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
movieId,,,,,,,,,,,,,,,
1,0.000549,0.004621,0.004532,0.001062,-0.003482,0.009426,-0.005740,3.998214,4.499908,0.082553,-0.003366,3.997779,0.027295,0.093120,-0.003467
5,-0.001363,2.992575,-0.011290,-0.001321,0.007495,-0.003411,-0.001639,-0.000376,0.000608,-0.147311,0.005043,0.003157,-0.009346,-0.013706,0.002704
6,0.003562,0.003606,-0.042919,0.001920,0.000583,0.018962,-0.006571,-0.001139,-0.000620,0.067985,-0.001920,-0.001631,-0.005073,0.089420,3.996265


In [ ]:
cnt=0
for i in new_movieratings.columns:
  if new_movieratings.loc[1][i]>=0:
    cnt+=1
    # print(new_movieratings.loc[1][i],i)
print(cnt)

210


In [ ]:
#Recommendations for user 1

def get_recommendations(df,user_id,ignore_ids,topn):
  df=df[user_id].sort_values(ascending=False).reset_index()
  print(df.head(4))
  dfans=df[~df['movieId'].isin(ignore_ids)].head(topn)
  
  print(dfans.head(topn))


get_recommendations(new_movieratings,1,movies_already_predictedby1,10)

   movieId         1
0      858  5.224232
1     1246  4.978352
2     2918  4.978025
3     4878  4.977303
    movieId         1
26      527  0.268651
27     1092  0.204086
28     4474  0.204086
29     3101  0.204086
30      480  0.201872
31     1097  0.198863
32     1270  0.179200
33      500  0.133527
34     2571  0.024718
35   168252  0.023731
